[View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/tensorflow/quantize_tensorflow_inference_inc.ipynb)

# Quantize Tensorflow Model for Inference using Intel Neural Compressor

With Intel Neural Compressor (INC) as quantization engine, you can apply `InferenceOptimizer.quantize` API to realize post-training quantization on your Tensorflow Keras models, which takes only a few lines.

To quantize your model with INC, the following dependencies need to be installed first:

In [ ]:
# for BigDL-Nano
!pip install --pre --upgrade bigdl-nano[tensorflow,inference]
!source bigdl-nano-init

> 📝 **Note**
> 
> We recommend to run the commands above, especially `source bigdl-nano-init` before jupyter kernel is started, or some of the optimizations may not take effect.

Let's take an [EfficientNetB0 model](https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB0) pretrained on ImageNet dataset and finetuned on [Imagenette](https://www.tensorflow.org/datasets/catalog/imagenette) dataset for validation as an example (seen full definition of `prepare_dataset` and `create_model` in [runnable example](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/tensorflow/quantize_tensorflow_inference_inc.ipynb)):

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from bigdl.nano.tf.keras import Model
from tensorflow.keras import layers
import tensorflow as tf

def create_model(num_classes=10, img_size=224):
    inputs = layers.Input(shape=(img_size, img_size, 3))
    x = tf.cast(inputs, tf.float32)
    backbone = EfficientNetB0(weights='imagenet', include_top=False)
    backbone.trainable = False
    x = backbone(x)
    x = layers.GlobalAveragePooling2D('channels_last')(x)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

And we obtain our training and testing dataset as follows. 

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

def prepare_dataset(img_size=224, batch_size=512):
    (ds_train, ds_test), ds_info = tfds.load(
        'imagenette/320px-v2',
        data_dir="./data/",
        split=['train', 'validation'],
        with_info=True,
        as_supervised=True
    )

    num_classes = ds_info.features['label'].num_classes

    def preprocessing(input_img, label):
        return tf.image.resize(input_img, (img_size, img_size)), tf.one_hot(label, num_classes)

    AUTOTUNE = tf.data.AUTOTUNE
    ds_train_batched = ds_train.shuffle(1000).map(preprocessing).batch(batch_size, drop_remainder=False).prefetch(AUTOTUNE)
    ds_test = ds_test.map(preprocessing).batch(batch_size, drop_remainder=False).prefetch(AUTOTUNE)
    calib_set = ds_train.map(preprocessing).prefetch(AUTOTUNE)

    return ds_train_batched, ds_test, calib_set, ds_info 

Then we finetune the model by following codes.

In [ ]:
train_set, test_set, calibration_set, ds_info = prepare_dataset()
ori_model = create_model()
ori_model.fit(train_set,
          epochs=10,
          steps_per_epoch=(ds_info.splits['train'].num_examples // 512 + 1),
          )

To enable quantization using INC for inference, you could simply **import BigDL-Nano** `InferenceOptimizer`**, and use** `InferenceOptimizer` **to quantize your TensorFlow model**:

In [ ]:
from bigdl.nano.tf.keras import InferenceOptimizer

q_model = InferenceOptimizer.quantize(ori_model, 
                                      x=calibration_set)

> 📝 **Note**
> 
>  `InferenceOptimizer` will by default quantize your TensorFlow models using int8 precision through **static** post-training quantization. Currently 'dynamic' approach is not supported yet. For this case, `x` (for calibration data) is required. To avoid data leak during calibration, it is suggested using training dataset or the subset of training set. 
> 
> Please refer to [API documentation](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/Nano/tensorflow.html) for more information on `InferenceOptimizer.quantize`.

You could then do the normal inference steps with the quantized model:

In [ ]:
x = tf.random.normal(shape=(2, 224, 224, 3))
# use the optimized model here
y_hat = q_model(x)
predictions = tf.argmax(y_hat, axis=1)
print(predictions)

> 📚 **Related Readings**
> 
> - [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/nano.html#install)
> - [How to install BigDL-Nano in Google Colab](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/install_in_colab.html)